## Agents with code generation


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from openai import AzureOpenAI
from helpers import function_to_schema

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
)

Data analysis agent

In [6]:
# Create an assistant
visualization_agent = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"), 
)

In [8]:
import time
import json
from IPython.display import clear_output

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=visualization_agent.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)
start_time = time.time()
status = run.status
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

Status: completed
Elapsed time: 0 minutes 12 seconds
assistant-k7B0YiqnT0vsIO04lxhgEHRA


Display generated code

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

In [10]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()